In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('parsedData/out.csv', index_col=0)
data

C:\Users\Shane\anaconda3\envs\py39\lib\site-packages\IPython\core\interactiveshell.py:3012: DtypeWarning: Columns (1,2,3,6,8,9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Shane\anaconda3\envs\py39\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Resident Status,Education,Month of Death,Sex,Age,Place of death and decedent's status,Marital Status,Day of Week of Death,Current Data Year,Injury at Work,Manner of Death,Method of Disposition,Autopsy,Activity Code,Place of injury,Underlying cause of death,Multiple Conditions,Race,Usual Occupation
0,1,61,1,M,1064 381808,4,S,5,2020,N,2,C,N,9,0,X74 429 125 40,0411S299 21S219 22X74 61T141 ...,01 11,101003939020
1,1,41,1,F,1039 331306,4,D,4,2020,N,2,C,N,9,0,X74 429 125 40,0411S069 21S019 22X74 61T141 ...,01 11,364911819016
2,1,31,1,F,1059 371708,4,S,5,2020,U,7,C,N,,,C139074 020 15,0411C139 61C845 62F109 63F179 ...,02 32,962022999023
3,1,31,1,M,1069 391909,2,M,4,2020,U,7,C,N,,,I219211 059 21,0311I490 12I469 13I219 ...,01 11,610018028001
4,1,31,1,M,1020 301004,4,S,4,2020,N,2,B,N,9,0,X74 429 125 40,0311S019 12X74 61T141 ...,03 23,380012947020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3390272,1,31,12,M,1071 402009,4,W,5,2020,U,7,C,N,,,I250214 062 21,0311I469 21I250 61B182 ...,02 32,470016579007
3390273,1,91,12,M,1060 381808,4,U,2,2020,U,7,B,N,,,E149159 046 16,0211E149 61I10 ...,02 132,990025999023
3390274,1,91,12,M,1067 391909,4,U,6,2020,U,7,B,N,,,B227051 016 03,0411C61 21B24 61I64 62B182 ...,02 132,990025999023
3390275,1,91,12,M,1054 361607,4,U,2,2020,U,7,B,N,,,N319331 111 37,0311I251 21N185 31N319 ...,02 132,


# Drop uneeded Columns

In [2]:
droppedCols = data.drop(['Resident Status', 'Month of Death', 'Place of death and decedent\'s status', 'Marital Status', 'Day of Week of Death', 'Current Data Year', 'Injury at Work',
       'Manner of Death', 'Method of Disposition', 'Autopsy', 'Activity Code',
       'Place of injury'], axis='columns')

In [3]:
print(droppedCols.columns)

Index(['Education', 'Sex', 'Age', 'Underlying cause of death',
       'Multiple Conditions', 'Race', 'Usual Occupation'],
      dtype='object')


# Decoding

### Sex

In [4]:
dropIndexes = []
for index, sex in enumerate(droppedCols['Sex']):
    if sex != 'M' and sex != 'F':
        dropIndexes.append(index)

print(len(dropIndexes))

# this also solves most of the errant values for other columns
cleanedSex = droppedCols.drop(index=dropIndexes)

245


### Age

In [5]:
# little exploration to see how relevant
monthCount = dayCount = hourCount = minuteCount = noAge = 0
dropIndexes = [1355654]
for index, age in enumerate(cleanedSex['Age']):
    try:
        float(age[1:4])
    except:
        dropIndexes.append(index)
        continue
    if age[0] == '2':
        monthCount +=1
    elif age[0] == '4':
        dayCount += 1
    elif age[0] == '5':
        hourCount += 1
    elif age[0] == '6':
        minuteCount += 1
    elif age[0] == '1':
        continue
    else:
        noAge+= 1
        dropIndexes.append(index)

cleanedAgeSex = cleanedSex.drop(index=cleanedSex.iloc[dropIndexes].index)
print(f'Months: {monthCount}\tDays: {dayCount}\t Hours: {hourCount}\t Minutes: {minuteCount}\tNo Age: {noAge}')

Months: 6739	Days: 5354	 Hours: 4878	 Minutes: 2674	No Age: 188


In [6]:

def decodeAge(age):
    if age[0] == '1':
        return float(age[1:4])
    elif age[0] == '2':
        return float(age[1:4])/12
    elif age[0] == '4':
        return float(age[1:4])/365
    elif age[0] == '5':
        return float(age[1:4])/8760
    elif age[0] == '6':
        return float(age[1:4])/525600
    else:
        
        raise Exception


decodedAge = cleanedAgeSex.copy()
decodedAge['Age'] = decodedAge['Age'].map(lambda age: decodeAge(age))

### Education

In [7]:
def decodeEducation(education):
    intCastEd = int(education)//10
    if intCastEd == 1:
        return "8th grade or less"
    elif intCastEd == 2:
        # dropped out of highschool
        return "HS no diploma"
    elif intCastEd == 3:
        return "HS or GED"
    elif intCastEd == 4:
        return "College no degree"
    elif intCastEd == 5:
        return "Associate's"
    elif intCastEd == 6:
        return "Bachelor's"
    elif intCastEd == 7:
        return "Master's"
    elif intCastEd == 8:
        return "Doctorate"
    else:
        raise Exception


unknownCount = 0
dropIndexes = []
for index, education in enumerate(decodedAge['Education']):
    castedEd = int(education)//10
    if castedEd >= 9:
        dropIndexes.append(index)
        unknownCount +=1


decodedEducation = decodedAge.copy().drop(index=decodedAge.iloc[dropIndexes].index)
decodedEducation['Education'] = decodedEducation['Education'].map(lambda education: decodeEducation(education))
print(unknownCount)



68315


### Top 13 Diseases only Decode

In [41]:
topX = 20
print(f'Total Disease Deaths: {len(decodedEducation)}\tDeaths by top {topX} diseases: {decodedEducation["Underlying cause of death"].value_counts()[0:topX].sum()}\tNum Unique: {len(decodedEducation["Underlying cause of death"].unique())}')
print(decodedEducation['Underlying cause of death'].value_counts()[0:topX])

Total Disease Deaths: 3321527	Deaths by top 20 diseases: 1651407	Num Unique: 4465
U071054 018   37     341696
I251215 063   21     166932
C349093 027   08     130518
G309189 052   17     125512
J449267 086   28     112836
F03 175 111   37     103310
I219211 059   21      98600
I250214 062   21      73978
I500230 067   22      63172
I64 238 070   24      52882
C259088 025   07      44898
C509104 029   09      42016
G20 188 051   37      39585
I119207 056   20      39221
J189257 078   27      38102
X42 420 122   39      38031
A419023 010   37      36990
C189081 023   06      36292
X44 420 122   39      34585
E149159 046   16      32251
Name: Underlying cause of death, dtype: int64


In [15]:
print(decodedEducation[decodedEducation.Age < 20 & decodedEducation['Underlying cause of death'].str.contains('U071')]["Underlying cause of death"].value_counts()[0:topX]) 
print(decodedEducation[decodedEducation.Age < 20]["Underlying cause of death"].value_counts())

Series([], Name: Underlying cause of death, dtype: int64)
X95 435 128   41     2567
P072358 10808933     2087
V892397 114   38     1502
R95 378 11013535     1245
X70 428 126   40     1111
                     ... 
I723243 074   26        1
C029073 020   15        1
P249362 10809933        1
I011199 055   22        1
K744300 095   30        1
Name: Underlying cause of death, Length: 1940, dtype: int64


In [43]:
icdCodeMap = {'U071': 'Covid-19', 'I251': 'Heart Disease', 'C349': 'Lung Cancer/Tumor', 'G309': 'Alzheimer\'s', 'X42 ': 'Drug Overdose', 'X70 ': 'Suicide',  
    'J449': 'COPD', 'F03 ': 'Dementia', 'I219': 'Heart Attack', 'I250': 'Heart Disease',
     'I500': 'Heart Failure', 'I64 ': 'Stroke', 'X44 ': 'Drug Overdose', 'X95 ': 'Shot', 'P072' : 'Car Accident'}


first4Death = decodedEducation['Underlying cause of death'].str.slice(0,4)

# print(test[~test.isna()][0:10])

decodedEducation['Cause of Death'] = first4Death.map(icdCodeMap)

decodedDeath = decodedEducation.drop(columns=['Underlying cause of death', 'Multiple Conditions'])
print(decodedDeath.size)


19929162


In [44]:
onlyMappedDeaths = decodedDeath[ ~decodedDeath['Cause of Death'].isna()]
print(onlyMappedDeaths['Cause of Death'].value_counts())
print(onlyMappedDeaths.columns)

Covid-19             341729
Heart Disease        240911
Lung Cancer/Tumor    130518
Alzheimer's          125512
COPD                 112836
Dementia             103310
Heart Attack          98606
Drug Overdose         72624
Heart Failure         63173
Stroke                52886
Shot                  17551
Suicide               12287
Car Accident           2090
Name: Cause of Death, dtype: int64
Index(['Education', 'Sex', 'Age', 'Race', 'Usual Occupation',
       'Cause of Death'],
      dtype='object')


### Usual Occupation

In [64]:
# TODO decode all of the usual occupation ones. (26 map)
print(onlyMappedDeaths[onlyMappedDeaths['Usual Occupation'].str.slice(4,6) == '  ']['Education'].value_counts())

occupationMap = {
    '01' : 'Management', '02' : 'Business/Financial', '03' : 'Computer/Math', '04' : 'Architecture/Engineering', '05' : 'Life/Physical/Social',
    '06' : 'Community/Social Services', '07' : 'Legal', '08' : 'Education', '09': 'Arts/Entertainment/Sports/Media', '10' : 'Healthcare Practitioner',
    '11' : 'Healthcare Support', '12' : 'Protective Service', '13' : 'Food Prep/Serving', '14' : 'Building Maintenance/Grounds Cleaning', '15':'Personal Care',
    '16' : 'Sales', '17' : 'Office/Administrative Support', '18' : 'Farming/Fishing/Forestry', '19' : 'Construction/Materials', '20' : 'Installation/Repair',
    '21' : 'Production', '22' : 'Transportation', '24' : 'Military', '25' : 'Other', '26' : 'Homemaker', '  ': 'None'
}
occupations = onlyMappedDeaths['Usual Occupation'].str.slice(4,6).map(occupationMap)

mappedOccupations = onlyMappedDeaths.copy()
mappedOccupations['Usual Occupation'] = occupations
finalOutput = mappedOccupations.drop(columns=['Race'])

HS or GED            47934
College no degree    13914
8th grade or less    13897
HS no diploma        13395
Bachelor's           10285
Associate's           7661
Master's              3983
Doctorate             1468
Name: Education, dtype: int64


## Total Dropped after Cleaning

In [67]:
finalOutput

finalOutput.to_csv('parsedData/cleaned.csv')